In [8]:
!pip install lifelines

     |████████████████████████████████| 358kB 7.7MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-cp37-none-any.whl size=4050 sha256=a7ffbfb20a9dd5c98f1fbd8dc3c9759ee25667b134a71012cfa01ea5f630984a
  Stored in directory: /root/.cache/pip/wheels/dc/68/dc/91321c55fba449755524481854f5be70d41912b8f886f908bb
Successfully built autograd-gamma


In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.utils.data as Data
from torch.utils.data import Dataset
import numpy as np
from lifelines.utils import concordance_index
import csv

In [2]:
class Regularization(object):
    def __init__(self, order, weight_decay):
        super(Regularization, self).__init__()
        self.order = order
        self.weight_decay = weight_decay

    def __call__(self, model):
        reg_loss = 0
        for name, w in model.named_parameters():
            if 'weight' in name:
                reg_loss = reg_loss + torch.norm(w, p=self.order)
        reg_loss = self.weight_decay * reg_loss
        return reg_loss

In [3]:
class DeepSurv(nn.Module):
    def __init__(self):
        super(DeepSurv, self).__init__()
        # parses parameters of network from configuration
        self.hidden_1 = nn.Linear(1046, 10)
        self.bat_1 = nn.BatchNorm1d(10)
        self.do_1 = nn.Dropout(0.5)
        self.hidden_2 = nn.Linear(10,5)
        self.bat_2 = nn.BatchNorm1d(5)
        self.do_2 = nn.Dropout(0.5)
        self.output = nn.Linear(5,1)

    def forward(self, x):
        x = self.do_1(nn.functional.selu(self.bat_1(self.hidden_1(x))))
        x = self.do_2(nn.functional.selu(self.bat_2(self.hidden_2(x))))
        x = self.output(x)
        return x

In [4]:
class NegativeLogLikelihood(nn.Module):
    def __init__(self):
        super(NegativeLogLikelihood, self).__init__()
        self.L2_reg = 0
        self.reg = Regularization(order=2, weight_decay=self.L2_reg)

    def forward(self, risk_pred, y, e, model):
        idx = y.sort(descending=True)[1]
        events = e[idx]
        risk_pred = risk_pred[idx]
        events = events.float()
        events = events.view(-1)
        risk_pred = risk_pred.view(-1)
        uncensored_likelihood = risk_pred - risk_pred.exp().cumsum(0).log()
        censored_likelihood = uncensored_likelihood * events
        num_observed_events = events.sum()
        neg_likelihood = -censored_likelihood.sum()/num_observed_events
        l2_loss = self.reg(model)
        return neg_likelihood + l2_loss

In [5]:
def c_index(risk_pred, y, e):
    if not isinstance(y, np.ndarray):
        y = y.detach().cpu().numpy()
    if not isinstance(risk_pred, np.ndarray):
        risk_pred = risk_pred.detach().cpu().numpy()
    if not isinstance(e, np.ndarray):
        e = e.detach().cpu().numpy()
    return concordance_index(y, risk_pred, e)

In [6]:
class Dataloader(Dataset):
  def __init__(self,filename,is_train):
      self.x,self.y,self.e = self.dataset_construction(filename,is_train)
  
  def dataset_construction(self, filename, is_train):
      with open(filename,'rt') as raw_data:
        data=np.loadtxt(raw_data,delimiter=',')

      if is_train:
        x = data[:60,3:].astype(np.float32)
        y = data[:60,0].astype(np.float32).reshape(-1,1)
        e = data[:60,1].reshape(-1,1)

      if not is_train:
        x = data[60:,3:].astype(np.float32)
        y = data[60:,0].astype(np.float32).reshape(-1,1)
        e = data[60:,1].reshape(-1,1)

      return x,y,e
  
  def __getitem__(self, item):
      
      x_item = self.x[item]
      e_item = self.e[item] 
      y_item = self.y[item] 
      
      x_tensor = torch.from_numpy(x_item)
      e_tensor = torch.from_numpy(e_item)
      y_tensor = torch.from_numpy(y_item)
      return x_tensor, y_tensor, e_tensor
  
  def __len__(self):
      return self.x.shape[0]

In [18]:
def train():
    model = DeepSurv()
    criterion = NegativeLogLikelihood()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    train_dataset = Dataloader(filename="ACC.csv",is_train=True)
    test_dataset = Dataloader(filename="ACC.csv",is_train=False)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=20)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=20)
    best_c_index = 0
    for epoch in range(0,500):
      model.train()
      for x,y,e in train_loader:
        risk_pred = model(x)
        #print(risk_pred)
        train_loss = criterion(risk_pred,y,e,model)
        #print(train_loss)
        train_c = c_index(-risk_pred,y,e)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
      
      model.eval()
      for x,y,e in test_loader:
        with torch.no_grad():
          risk_pred = model(x)
          valid_loss = criterion(risk_pred,y,e,model)
          valid_c = c_index(-risk_pred,y,e)
          #print(valid_c)
          if best_c_index < valid_c:
            best_c_index = valid_c
        
    return best_c_index

In [21]:
result=train()
result

0.45901639344262296